<a href="https://colab.research.google.com/github/siddhiagarwal029/Batch-3-Test/blob/master/Satellite_Imagery_Analysis_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gauravduttakiit_satellite_imagery_path = kagglehub.dataset_download('gauravduttakiit/satellite-imagery')

print('Data source import complete.')


100%|██████████| 35.9M/35.9M [00:00<00:00, 106MB/s] 

Extracting files...


Data source import complete.


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d gauravduttakiit/satellite-imagery-analysis-main


403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/gauravduttakiit/satellite-imagery-analysis-main


In [ ]:
from glob import glob

In [ ]:
pip install earthpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 87.6 MB/s eta 0:00:00


In [ ]:
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [ ]:
import rasterio as rio

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

In [ ]:
import plotly.graph_objects as go

In [ ]:
import os
print(os.getcwd())
print(os.listdir())



/content
['.config', 'sample_data']


In [ ]:
from glob import glob
S_sentinel_bands = glob("../input/satellite-imagery/Satellite_Imagery_Analysis-main/Data/sundarbans_data/*B?*.tiff")
print("Found files:", len(S_sentinel_bands))
print(S_sentinel_bands)


Found files: 0
[]


In [ ]:
!ls ../input/satellite-imagery/Satellite_Imagery_Analysis-main/Data/sundarbans_data


ls: cannot access '../input/satellite-imagery/Satellite_Imagery_Analysis-main/Data/sundarbans_data': No such file or directory


# Read Data

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
S_sentinel_bands = glob("../input/satellite-imagery/Satellite_Imagery_Analysis-main/Data/sundarbans_data/*B?*.tiff")
S_sentinel_bands.sort()

# Visualize Bands

In [ ]:
l = []
for i in S_sentinel_bands:
    with rio.open(i, 'r') as f:
        l.append(f.read(1))
arr_st = np.stack(l)
ep.plot_bands(arr_st, cmap = 'gist_earth', figsize = (20, 12), cols = 6, cbar = False)
plt.show()

ValueError: need at least one array to stack

In [ ]:
from glob import glob
import rasterio as rio
import numpy as np
import earthpy.plot as ep
import matplotlib.pyplot as plt

# Ensure the path is correct relative to your notebook's location
S_sentinel_bands = glob("../input/satellite-imagery/Satellite_Imagery_Analysis-main/Data/sundarbans_data/*B?*.tiff")
S_sentinel_bands.sort()

l = []
for i in S_sentinel_bands:
    # Check if the file exists before opening
    if os.path.exists(i):
        with rio.open(i, 'r') as f:
            l.append(f.read(1))
    else:
        print(f"File not found: {i}")  # Print an error message if file not found

# Check if l is empty before stacking
if l:
    arr_st = np.stack(l)
    ep.plot_bands(arr_st, cmap='gist_earth', figsize=(20, 12), cols=6, cbar=False)
    plt.show()
else:
    print("No data found in the specified files.")

No data found in the specified files.


# RGB Composite Image

In [ ]:
#RGB Composite Image

rgb = ep.plot_rgb(arr_st,
                  rgb=(3,2,1),
                  figsize=(10, 16))
plt.show()



NameError: name 'arr_st' is not defined

In [ ]:
# RGB Composite Image with Strech

ep.plot_rgb(arr_st,
            rgb=(3, 2, 1),
            stretch=True,
            str_clip=0.2,
            figsize=(10, 16))
plt.show()
ep.plot_rgb(
    arr_st,
    rgb=(3, 2, 1),
    stretch=True,
    str_clip=0.2,
    figsize=(10, 16),
    # title="RGB Composite Image with Stretch Applied",
)

plt.show()

In [ ]:
# Histograms

# Histograms

In [ ]:
colors = ['tomato', 'navy', 'MediumSpringGreen', 'lightblue', 'orange', 'blue',
          'maroon', 'purple', 'yellow', 'olive', 'brown', 'cyan']
ep.hist(arr_st,
         colors = colors,
        title=[f'Band-{i}' for i in range(1, 13)],
        cols=3,
        alpha=0.5,
        figsize = (12, 10)
        )

plt.show()

In [ ]:
ndvi = es.normalized_diff(arr_st[7], arr_st[3])
ep.plot_bands(ndvi, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))
plt.show()

# Vegetation and Soil Indices

## Normalized Difference Vegetation Index (NDVI)

To determine the density of green on a patch of land, researchers must observe the distinct colors (wavelengths) of visible(VIS) and near-infrared (NIR)sunlight reflected by the plants. The Normalized Difference Vegetation Index (NDVI) quantifies vegetation by measuring the difference between near-infrared which vegetation strongly reflects and red light (which vegetation absorbs). NDVI always ranges from -1 to +1.

In [ ]:
ndvi = es.normalized_diff(arr_st[7], arr_st[3])

ep.plot_bands(ndvi, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()

# Soil Adjusted Vegetation Index (SAVI)

The Soil-Adjusted Vegetation Index (SAVI) is a vegetation index that attempts to minimize soil brightness influences using a soil-brightness correction factor. This is often used in arid regions where vegetative cover is low.

In [ ]:
L = 0.5

savi = ((arr_st[7] - arr_st[3]) / (arr_st[7] + arr_st[3] + L)) * (1 + L)

ep.plot_bands(savi, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()

# Visible Atmospherically Resistant Index (VARI)

The Visible Atmospherically Resistant Index (VARI) is designed to emphasize vegetation in the visible portion of the spectrum while mitigating illumination differences and atmospheric effects. It is ideal for RGB or color images; it utilizes all three color bands.

In [ ]:
vari = (arr_st[2] - arr_st[3])/ (arr_st[2] + arr_st[3] - arr_st[1])

ep.plot_bands(vari, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()

# Water Indices

Surface water change is a very important indicator of environmental, climatic, and anthropogenic activities. Remote sensors, such as sentinel-2, Landsat, have been providing data for the last four decades, which are useful for extracting land cover types such as forest and water. Researchers have proposed many surface water extraction techniques, among which index-based methods are popular owing to their simplicity and cost-effectiveness.

## Modified Normalized Difference Water Index (MNDWI)

The Modified Normalized Difference Water Index (MNDWI) uses green and SWIR bands for the enhancement of open water features. It also diminishes built-up area features that are often correlated with open water in other indices.

In [ ]:
mndwi = es.normalized_diff(arr_st[2], arr_st[10])

ep.plot_bands(mndwi, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()

## Normalized Difference Moisture Index (NDMI)

The Normalized Difference Moisture Index (NDMI) is sensitive to the moisture levels in vegetation. It is used to monitor droughts as well as monitor fuel levels in fire-prone areas. It uses NIR and SWIR bands to create a ratio designed to mitigate illumination and atmospheric effects.

In [ ]:
ndmi = es.normalized_diff(arr_st[7], arr_st[10])

ep.plot_bands(ndmi, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()

# Geology Indices

Satellite imagery and aerial photography have proven to be important tools in support of mineral exploration projects. They can be used in a variety of ways. Firstly they provide geologists and field crews the location of tracks, roads, fences, and inhabited areas.

## Clay Minerals

The clay ratio is a ratio of the SWIR1 and SWIR2 bands. This ratio leverages the fact that hydrous minerals such as clays, alunite absorb radiation in the 2.0–2.3 micron portion of the spectrum. This index mitigates illumination changes due to terrain since it is a ratio.

In [ ]:
cmr = np.divide(arr_st[10], arr_st[11])

ep.plot_bands(cmr, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()

## Ferrous Minerals

The ferrous minerals ratio highlights iron-bearing materials. It uses the ratio between the SWIR band and the NIR band.

In [ ]:
fmr = np.divide(arr_st[10], arr_st[7])

ep.plot_bands(fmr, cmap="RdYlGn", cols=1, vmin=-1, vmax=1, figsize=(10, 14))

plt.show()